# Building a Banking Customer Service Chatbot with LanceDB and Ollama

In this notebook, we are building a customer service chatbot for banking using LanceDB and Ollama. The chatbot is designed to provide efficient and professional responses to customer inquiries.

We are using the **Bitext Retail Banking LLM Chatbot Training Dataset** from Hugging Face to train and set up a Retrieval-Augmented Generation (RAG) system. This dataset contains instructions, intents, responses, and other relevant fields to help the chatbot understand and respond to user queries effectively.

The workflow includes:
1. Loading the Bitext dataset.
2. Embedding the data using Ollama's embedding model.
3. Storing the data in LanceDB with a defined schema.
4. Setting up a RAG pipeline to retrieve relevant context for user queries.
5. Using the retrieved context to generate accurate and empathetic responses.

This approach ensures the chatbot is both contextually aware and capable of handling a wide range of banking-related queries.

In [2]:
import ollama
import pandas as pd
import lancedb
import time
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import EmbeddingFunctionRegistry
from utils import view

In [3]:
df = pd.read_csv("data/bitext-retail-banking-llm-chatbot-training-dataset.csv")
# download data from https://huggingface.co/datasets/bitext/Bitext-retail-banking-llm-chatbot-training-dataset/tree/main

In [28]:
df.iloc[0]

tags                                                       BCIPZ
instruction     I would like to acivate a card, can you help me?
category                                                    CARD
intent                                             activate_card
response       I'm here to assist you with that! Activating y...
Name: 0, dtype: object

In [5]:
registry = EmbeddingFunctionRegistry.get_instance()
embedder = registry.get("ollama").create(name="mxbai-embed-large")

In [6]:
class DataSchema(LanceModel):
  instruction: str = embedder.SourceField()
  vector: Vector(embedder.ndims()) = embedder.VectorField()
  category: str
  intent: str
  response: str
  tags:str

In [8]:
db = lancedb.connect("./lancedb")

query_table = db.create_table("queries", schema=DataSchema,mode="overwrite")

In [9]:
%%time
query_table.add(df)

CPU times: user 48.8 s, sys: 7.33 s, total: 56.2 s
Wall time: 16min 11s


In [29]:
query_table.to_pandas().iloc[0]

instruction     I would like to acivate a card, can you help me?
vector         [-0.29672346, -0.24280721, -0.32037482, 0.2291...
category                                                    CARD
intent                                             activate_card
response       I'm here to assist you with that! Activating y...
tags                                                       BCIPZ
Name: 0, dtype: object

In [31]:
question = "My card is not working"
rows = (query_table
  .search(question)
  .limit(5)
  .to_pydantic(DataSchema)
)

view(rows)

[
    DataSchema(
        instruction='I got to activaste a card, could I get some help?',
        vector=FixedSizeList(dim=1024),
        category='CARD',
        intent='activate_card',
        response='I\'m here to assist you with activating your card. Activating a card is a simple process. Please 
follow the steps below:\n\n1. Visit our website at {{Company Website URL}}.\n2. Log in to your account using your 
credentials. If you don\'t have an account yet, you can create one by clicking on the "Sign Up" button.\n3. Once 
you\'re logged in, navigate to the "Cards" or "Account" section.\n4. Look for the option to activate your card. It 
might be labeled as "Activate Card" or something similar.\n5. Click on the activation link and follow the prompts 
to complete the activation process.\n6. If you encounter any difficulties or have any questions along the way, feel
free to reach out to our customer support team. They are available {{Customer Support Working Hours}} at {{Customer
Supp

In [32]:
def transform_data(rows):
  return [
    {"instruction": r.instruction, "category": r.category, "intent": r.intent,"response": r.response,"tags":r.tags}  
    for r in rows
  ]

In [33]:
view(transform_data(rows))

[
    {
        'instruction': 'I got to activaste a card, could I get some help?',
        'category': 'CARD',
        'intent': 'activate_card',
        'response': 'I\'m here to assist you with activating your card. Activating a card is a simple process. 
Please follow the steps below:\n\n1. Visit our website at {{Company Website URL}}.\n2. Log in to your account using
your credentials. If you don\'t have an account yet, you can create one by clicking on the "Sign Up" button.\n3. 
Once you\'re logged in, navigate to the "Cards" or "Account" section.\n4. Look for the option to activate your 
card. It might be labeled as "Activate Card" or something similar.\n5. Click on the activation link and follow the 
prompts to complete the activation process.\n6. If you encounter any difficulties or have any questions along the 
way, feel free to reach out to our customer support team. They are available {{Customer Support Working Hours}} at 
{{Customer Support Phone Number}} or through the Liv

In [16]:
SYSTEM = """
You are a professional, friendly, and knowledgeable customer service representative for Anies Bank, a retail banking institution. Your goal is to assist users efficiently while maintaining a warm and approachable tone. Provide clear, concise, and accurate responses to customer inquiries, ensuring their concerns are addressed with empathy and professionalism.

Guidelines:  
- Be polite and welcoming, using a professional yet friendly tone.  
- Provide factually accurate and easy-to-understand answers.  
- Assist with common banking queries such as account setup, transaction disputes, loan inquiries, security concerns, and payment issues.  
- Acknowledge complaints with empathy and offer solutions or escalate when necessary.  
- Never request sensitive information like passwords or PINs. Instead, guide users to official security procedures.  
- Escalate complex issues when required and set clear response time expectations.

Example responses:  

User: "I was charged twice for a transaction. What should I do?"  
AI: "I’m sorry for the trouble! If you've been charged twice, please check your transaction history. If it's a duplicate, you can dispute it in [Dispute Center]. Let me know if you need further assistance!"  

User: "I think someone hacked my account!"  
AI: "That sounds serious! Please reset your password immediately and enable two-factor authentication. If you notice unauthorized transactions, contact our fraud team at [support@email.com] or [hotline]. Let me know if you need more help!"  

User: "How can I apply for a loan?"  
AI: "You can apply for a loan by visiting the 'Loans' section in our mobile app or website. You'll find all the details about loan types, interest rates, and eligibility criteria. Let me know if you need help with the application process!"  

"""

In [34]:
while True:
  message = input("Message: ")
  if message.lower() == "exit":
    break
  rows = (query_table.search(message).limit(20).to_pydantic(DataSchema))
  context = transform_data(rows)
  stream = ollama.chat(
    model="llama3.2", stream=True,
    messages = [
      { "role": "system", 'content': SYSTEM},
      { "role": "user", 'content': f"Here is the relevant context for your query: {context}"},
      { "role": "user", 'content': f"My question is: {message}"}
    ]
  )
  for chunk in stream:
    print(chunk['message']['content'], end='', flush=True)
  print("\n===============\n")

I'm so sorry to hear that you're having trouble logging in! Can you please tell me a bit more about what's happening? Was there an error message, or did your account lock up on you?

Also, just to make sure, I want to remind you that we don't store passwords in plain text here. If you've forgotten your password, the best way to reset it is by using the 'Forgot Password' option on our login page.

Would you like me to walk you through the steps to reset your password or help you troubleshoot what might be going on?



TypeError: Unsupported query type: <class 'NoneType'>